In [2]:
!pip install camb

In [2]:
import os
import numpy as np
import camb
from camb.dark_energy import DarkEnergyPPF, DarkEnergyFluid
from tqdm import tqdm

def generate_camb_cl(H0=67.5, ombh2=0.022, omch2=0.122, mnu=0.06, omk=0,
                     tau=0.06, logA=3.05, ns=0.965, r=0, w = -1, wa = 0):
    """ Generate a temperature power spectrum given cosmological parameters"""
    As = 1e-10 * np.exp(logA)
    pars = camb.CAMBparams()
    pars.set_cosmology(H0=H0, ombh2=ombh2, omch2=omch2, mnu=mnu, omk=omk, tau=tau)
    pars.InitPower.set_params(As=As, ns=ns, r=r)
    if (w != -1 or wa != 0):
        pars.DarkEnergy = DarkEnergyPPF(w=w, wa=wa)
    pars.set_for_lmax(2500, lens_potential_accuracy=0);
    pars.NonLinear = camb.model.NonLinear_none
    results = camb.get_results(pars)
    powers = results.get_cmb_power_spectra(pars, CMB_unit='muK')
    totCL = powers['total']
    return totCL[2:, 0]

def generate_sims(N, mins=None, maxs=None, save=True):
    """ Generated N randomly generates CLs """

    if maxs is None:
        maxs = np.array([90, 0.05, 0.5, 3.5, 1])
    if mins is None:
        mins = np.array([50, 0.01, 0.01, 1.5, 0.8])

    cls = np.zeros([N, 2549])
    H0_rand = np.random.uniform(mins[0], maxs[0], N)
    ombh2_rand = np.random.uniform(mins[1], maxs[1], N)
    omch2_rand = np.random.uniform(mins[2], maxs[2], N)
    logA_rand = np.random.uniform(mins[3], maxs[3], N)
    ns_rand = np.random.uniform(mins[4], maxs[4], N)

    skipped = []
    for i in tqdm(range(N)):
        if i % 500 == 0: print('Generated ', i, 'out of ', N)

        try:
            cls[i] = generate_camb_cl(H0=H0_rand[i],
                                      ombh2=ombh2_rand[i],
                                      omch2=omch2_rand[i],
                                      logA=logA_rand[i],
                                      ns=ns_rand[i])
        except:
            print("Skipped iteration ", i)
            skipped.append(i)

    params = np.stack([H0_rand,
                       ombh2_rand,
                       omch2_rand,
                       logA_rand,
                       ns_rand]).T

    params = np.delete(params, skipped, axis=0)

    if save:
        try:
            cls_saved = np.load('cls.npy')
            cls_saved = cls_saved.tolist()
            params_saved = np.load('params.npy')
            params_saved = params_saved.tolist()
            print('Saved files are used')
        except:
            cls_saved = []
            params_saved = []
            print('Generating a new file')
        for i in range(len(cls)):
            cls_saved.append(cls[i])
            params_saved.append(params[i])

        np.save('cls.npy', cls_saved)
        np.save('params.npy', params_saved)

    return cls, params

In [3]:
    N = 10000
    cls, params = generate_sims(N=N, save=True)
    print('COMPLETED')

 15%|█▍        | 1463/10000 [28:22<2:40:49,  1.13s/it]

Generated  0 out of  10000
Generated  500 out of  10000
Generated  1000 out of  10000


In [1]:
import pylab as plt
cls_file = np.load('cls.npy')
for i in range(len(cls_file)):
    plt.plot(cls_file[i])
print (len(cls_file))

NameError: name 'np' is not defined

In [23]:
mylist = [[1, 2, 3, 4], [4, 5, 6, 7]]
new = [[7, 8, 9, 0], [2, 3, 4, 5]]
mylist.append(new)
print(mylist)

[[1, 2, 3, 4], [4, 5, 6, 7], [[7, 8, 9, 0], [2, 3, 4, 5]]]
